In [ ]:
#Section A
import cv2
img = cv2.imread(r"YOLOimages\city.jpg",1) #Reading the image
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #Converting to grayscale
blurred = cv2.GaussianBlur(gray,(3,3),0) #Blurring
edge = cv2.Canny(blurred, 100, 200, 5) #Detecting edges
w,h=img.shape[:2]
Ms = cv2.getRotationMatrix2D((w//2,h//2),0.0,2.0) #scale by 2x
Mr = cv2.getRotationMatrix2D((w//2,h//2),39.0,1.0) #rotate by 39 degrees
Mrs = cv2.getRotationMatrix2D((w//2,h//2),43.0,3.5) #rotate by 43 degrees and scale by 3.5x
rotated = cv2.warpAffine(img, Mr, (w, h)) #rotated image
scaled = cv2.warpAffine(img, Ms, (w, h)) #scaled image
rot_n_scaled = cv2.warpAffine(img, Mrs, (w, h)) #applying both rotation and scaling
#Displaying images
cv2.imshow('Original',img)
cv2.imshow('Grayed',gray)
cv2.imshow('Blurred',blurred)
cv2.imshow('Edges',edge)
cv2.imshow('Rotated',rotated)
cv2.imshow('Scaled',scaled)
cv2.imshow('Rotated and Scaled',rot_n_scaled)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#Section B
import tensorflow as tf
#importing necessary functions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
#load mnist dataset
(i_train,o_train),(i_test,o_test)=tf.keras.datasets.mnist.load_data()
#normalise 0-255 to 0-1
i_train=i_train/255.0
i_test=i_test/255.0
#reshape to match the input parameter dimensions for Conv2D function
i_train=i_train.reshape(-1,28,28,1)
i_test=i_test.reshape(-1,28,28,1) 
"""
28,28 because the data loaded from mnist dataset has the dimensions 28,28 if we want it 
to be different we can resize the image beforehand after loading
"""
#labels to numbers(one-hot encoding)
o_train=to_categorical(o_train,num_classes=10)
o_test=to_categorical(o_test,num_classes=10)
#split loaded training data into training and validation sets with test size being random 20%
i_train,i_val,o_train,o_val=train_test_split(i_train,o_train,test_size=0.2,random_state=42)
#we assign a seed(random state) so that the split is the same, 
# if split changes everytime then the diff in performnce could be due to different split rather than the changes made
#model definition
model=Sequential([
    Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)),#32 features, 3x3 kernel, activatioin mode is set to relu
    #relu outputs zero for -ve inputs and out=in for +ve inputs
    MaxPooling2D((2,2)), #Taking maximum value from 2x2 grid
    #now the shape is (14,14)
    Conv2D(64,(3,3),activation='relu'),#covolution with 64 features and same kernel size and activation method
    MaxPooling2D((2,2)), #Maxpooling again
    Flatten(), #Flatten into a 1D array
    Dense(128,activation='relu'), #reduce the no. of element feature vectors to 128
    Dropout(0.4), #randomly drops 40% of the 128 outputs to prevent overfitting
    Dense(10, activation='softmax') #10 to match the number of output classes in MNIST dataset
])
#compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
#training the model
history=model.fit(
    i_train,o_train,
    validation_data=(i_val,o_val),
    epochs=10,
    batch_size=32
)
#Evaluation of loss and accuracy after training
testloss,testaccu=model.evaluate(i_test,o_test,verbose=2)
print(f"Test Accuracy:{testaccu:.4f}")
#plotting  Accuracy vs Epochs and Loss vs Epochs
plt.figure(figsize=(12, 4))
#Accuracy
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'],label='Training Accuracy')
plt.plot(history.history['val_accuracy'],label='Validation Accuracy')
plt.title('Accuracy vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
#Loss
plt.subplot(1,2,2)
plt.plot(history.history['loss'],label='Training Loss')
plt.plot(history.history['val_loss'],label='Validation Loss')
plt.title('Loss vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')


plt.tight_layout()
plt.show()
#Testing and Visualising the prediction
predictions=model.predict(i_test[:20])#the number can be increased by choice

fig, axes=plt.subplots(2,10,figsize=(20,5))
axes=axes.flatten()
for i,ax in enumerate(axes):
    ax.imshow(i_test[i].reshape(28,28),cmap='gray')
    ax.set_title(f"Pred:{np.argmax(predictions[i])}\nTrue:{np.argmax(o_test[i])}")
    ax.axis('off')
plt.tight_layout()
plt.show()
    

In [ ]:
#Section C
#import necessary modules
import cv2
import torch
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np
import os

model = YOLO("yolov8n.pt")#Pre-trained YOLO model
image_paths = [
    "YOLOimages/room.jpg",
    "YOLOimages/traffic.jpg",
    "YOLOimages/study.jpg"
]
output_dir="YOLOimages"
for images in image_paths:
    img=cv2.imread(images)
    if img is None:
        print(f"Image not found")
        continue
    print(f"Processing:{images}")
    print(f"Image Size:{img.shape}")
    
    results=model(img)
    for result in results:
        boxes=result.boxes.xyxy.cpu().numpy()
        labels=result.boxes.cls.cpu().numpy()
        confscore=result.boxes.conf.cpu().numpy()
        
        print(f"Detected {len(boxes)} objects")
        
        for i, box in enumerate(boxes):
            x1,y1,x2,y2=map(int,box)
            label=model.names[int(labels[i])]
            confi=confscore[i]
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.putText(img,f"{label}:{confi:.3f}", (x1,y1-15),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),2)
            print(f"Object {i}:{label} at ({x1},{y1},{x2},{y2})\n Confidence Score:{confi:.3f}")
            
            output_path=os.path.join(output_dir,f"detected_{os.path.basename(images)}")
            cv2.imwrite(output_path,img)
            print(f"Outout file saved at:{output_path}")
            
new_images=[
    "YOLOimages/detected_room.jpg",
    "YOLOimages/detected_study.jpg",
    "YOLOimages/detected_traffic.jpg"
]
for images in new_images:
    img=cv2.imread(images)
    cv2.imshow('Detection Results', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
print("We are done!")